In [11]:
# --- Install required packages ---
!pip install -q kagglehub tensorflow

# --- Import libraries ---
import kagglehub
from pathlib import Path

# --- Download the dataset from KaggleHub ---
path = kagglehub.dataset_download("tushar5harma/plant-village-dataset-updated")
print(" Dataset path:", path)

# --- Check folder hierarchy ---
base = Path(path)
print("\nTop-level folders:")
for plant_dir in base.iterdir():
    if plant_dir.is_dir():
        print(" -", plant_dir.name)

# --- Inspect subfolders inside one example plant (Apple) ---
example_plant = base / "Apple"
print("\nSubfolders inside 'Apple':")
for sub in example_plant.iterdir():
    print(" -", sub.name)
    # Show one level deeper to see where images really are
    for sub2 in sub.iterdir():
        print("    ├──", sub2.name)
        break
    break


Using Colab cache for faster access to the 'plant-village-dataset-updated' dataset.
 Dataset path: /kaggle/input/plant-village-dataset-updated

Top-level folders:
 - Tomato
 - Apple
 - Bell Pepper
 - Strawberry
 - Corn (Maize)
 - Peach
 - Grape
 - Cherry
 - Potato

Subfolders inside 'Apple':
 - Val
    ├── Healthy


In [14]:
import os, shutil
from pathlib import Path

# Paths
raw_path = Path("/kaggle/input/plant-village-dataset-updated")
base_output = Path("/content/dataset_combined")
train_out, val_out, test_out = [base_output / s for s in ["train", "val", "test"]]

# Make new folders
for folder in [train_out, val_out, test_out]:
    folder.mkdir(parents=True, exist_ok=True)

# Loop through all plants
for plant_dir in raw_path.iterdir():
    if not plant_dir.is_dir() or plant_dir.name.startswith('.'):
        continue

    print(f"\nProcessing {plant_dir.name} ...")

    for subset in ["Train", "Val", "Test"]:
        subset_dir = plant_dir / subset
        if not subset_dir.exists():
            continue

        # Each class folder (like Healthy, Apple Scab, etc.)
        for class_dir in subset_dir.iterdir():
            if not class_dir.is_dir() or class_dir.name.startswith('.'):
                continue

            # Create a unique class name like "Apple___Apple_Scab"
            class_name = f"{plant_dir.name}___{class_dir.name.replace(' ', '_')}"
            dest_dir = base_output / subset.lower() / class_name
            dest_dir.mkdir(parents=True, exist_ok=True)

            # Copy all images safely
            for img_file in class_dir.glob("*"):
                if img_file.is_file():  # Skip directories like .ipynb_checkpoints
                    shutil.copy(img_file, dest_dir)

print("\n Dataset successfully merged into:")
print(base_output)

# Check resulting structure
for subset in ["train", "val", "test"]:
    subset_path = base_output / subset
    classes = [f.name for f in subset_path.iterdir() if f.is_dir()]
    print(f"\n📁 {subset.upper()} set: {len(classes)} classes")
    print("Sample classes:", classes[:5])



Processing Tomato ...

Processing Apple ...

Processing Bell Pepper ...

Processing Strawberry ...

Processing Corn (Maize) ...

Processing Peach ...

Processing Grape ...

Processing Cherry ...

Processing Potato ...

 Dataset successfully merged into:
/content/dataset_combined

📁 TRAIN set: 29 classes
Sample classes: ['Apple___Apple_Scab', 'Cherry___Healthy', 'Cherry___Powdery_Mildew', 'Potato___Healthy', 'Bell Pepper___Bacterial_Spot']

📁 VAL set: 29 classes
Sample classes: ['Apple___Apple_Scab', 'Cherry___Healthy', 'Cherry___Powdery_Mildew', 'Potato___Healthy', 'Bell Pepper___Bacterial_Spot']

📁 TEST set: 29 classes
Sample classes: ['Apple___Apple_Scab', 'Cherry___Healthy', 'Cherry___Powdery_Mildew', 'Potato___Healthy', 'Bell Pepper___Bacterial_Spot']


In [5]:
plants = [p for p in Path(dataset_dir).iterdir() if p.is_dir()]
MAX_IMAGES = 1500  # limit per class

for plant in plants:
    print(f"\nProcessing plant: {plant.name}")
    for subset in ['train', 'test', 'val']:
        subset_path = plant / subset
        if not subset_path.exists():
            continue

        for category in subset_path.iterdir():  # e.g. 'Apple Scab', 'Healthy'
            if not category.is_dir():
                continue

            class_name = f"{plant.name}_{category.name}".replace(" ", "_")
            output_folder = base_output / subset / class_name
            output_folder.mkdir(parents=True, exist_ok=True)

            all_images = list(category.glob("*.jpg")) + list(category.glob("*.png"))
            random.shuffle(all_images)
            selected = all_images[:MAX_IMAGES]

            for img_path in tqdm(selected, desc=f"{subset}/{class_name}", leave=False):
                shutil.copy(img_path, output_folder / img_path.name)



Processing plant: Tomato

Processing plant: Apple

Processing plant: Bell Pepper

Processing plant: Strawberry

Processing plant: Corn (Maize)

Processing plant: Peach

Processing plant: Grape

Processing plant: Cherry

Processing plant: Potato


In [8]:
import os
import shutil
from pathlib import Path
from tqdm import tqdm
import random

# Paths
source_path = Path("/kaggle/input/plant-village-dataset-updated")
output_path = Path("/content/dataset_final")

# Delete previous folder
if output_path.exists():
    shutil.rmtree(output_path)
print("Deleted previous dataset_final folder...")

# Create new structure
for subset in ["train", "val", "test"]:
    (output_path / subset).mkdir(parents=True, exist_ok=True)

TRAIN_COUNT = 1500
VAL_COUNT = 225

# Process each plant
for plant_dir in tqdm(list(source_path.iterdir()), desc="Processing plants"):
    if not plant_dir.is_dir():
        continue
    print(f"\nProcessing plant: {plant_dir.name}")

    # Each plant has train/val/test subfolders
    for class_dir in plant_dir.glob("**/*"):
        if not class_dir.is_dir():
            continue

        class_name = f"{plant_dir.name}___{class_dir.name.replace(' ', '_')}"

        # Collect all images from train, val, test
        images = []
        for split in ["Train", "Val", "Test"]:
            split_path = class_dir.parent / split / class_dir.name
            if split_path.exists():
                images += list(split_path.glob("*.*"))

        random.shuffle(images)

        # Split
        train_imgs = images[:TRAIN_COUNT]
        val_imgs = images[TRAIN_COUNT:TRAIN_COUNT+VAL_COUNT]
        test_imgs = images[TRAIN_COUNT+VAL_COUNT:]

        # Create folders
        for split_name in ["train", "val", "test"]:
            (output_path / split_name / class_name).mkdir(parents=True, exist_ok=True)

        # Copy images
        for img in train_imgs:
            shutil.copy(img, output_path / "train" / class_name / img.name)
        for img in val_imgs:
            shutil.copy(img, output_path / "val" / class_name / img.name)
        for img in test_imgs:
            shutil.copy(img, output_path / "test" / class_name / img.name)

        print(f"{class_name}: train={len(train_imgs)}, val={len(val_imgs)}, test={len(test_imgs)}")

print("\n✅ Dataset restructuring complete!")


Processing plants: 100%|██████████| 9/9 [00:00<00:00, 7371.36it/s]


Processing plant: Tomato

Processing plant: Apple

Processing plant: Bell Pepper

Processing plant: Strawberry

Processing plant: Corn (Maize)

Processing plant: Peach

Processing plant: Grape

Processing plant: Cherry

Processing plant: Potato

✅ Dataset restructuring complete!


In [58]:
from pathlib import Path

# Path to your new dataset
dataset_dir = Path("/content/dataset_final")

# Iterate through each split
for split in ["train", "val", "test"]:
    print(f"\n📂 {split.upper()} set:")
    split_dir = dataset_dir / split

    # Iterate through class folders
    for class_folder in sorted(split_dir.iterdir()):
        if not class_folder.is_dir():
            continue

        # Count images in the class folder
        num_images = len(list(class_folder.glob("*.*")))
        print(f"{class_folder.name}: {num_images} images")



📂 TRAIN set:
Apple___Apple_Scab: 1500 images
Apple___Black_Rot: 1500 images
Apple___Cedar_Apple_Rust: 1500 images
Apple___Healthy: 1500 images
Bell Pepper___Bacterial_Spot: 1500 images
Bell Pepper___Healthy: 1500 images
Cherry___Healthy: 1500 images
Cherry___Powdery_Mildew: 1500 images
Corn (Maize)___Cercospora_Leaf_Spot: 1500 images
Corn (Maize)___Common_Rust_: 1500 images
Corn (Maize)___Healthy: 1500 images
Corn (Maize)___Northern_Leaf_Blight: 1500 images
Grape___Black_Rot: 1500 images
Grape___Esca_(Black_Measles): 1500 images
Grape___Healthy: 1500 images
Grape___Leaf_Blight: 1500 images
Peach___Bacterial_Spot: 1500 images
Peach___Healthy: 1500 images
Potato___Early_Blight: 1500 images
Potato___Healthy: 1500 images
Potato___Late_Blight: 1500 images
Strawberry___Healthy: 1500 images
Strawberry___Leaf_Scorch: 1500 images
Tomato___Bacterial_Spot: 1500 images
Tomato___Early_Blight: 1500 images
Tomato___Healthy: 1500 images
Tomato___Late_Blight: 1500 images
Tomato___Septoria_Leaf_Spot: 1

In [61]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV3Large
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.applications.mobilenet_v3 import preprocess_input
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from pathlib import Path


In [62]:
BATCH_SIZE = 64
IMG_SIZE = (224, 224)
DATASET_DIR = Path("/content/dataset_final")  # Your dataset folder with train/val/test subfolders


In [63]:
# Load train dataset first to extract class names
train_ds_raw = image_dataset_from_directory(
    DATASET_DIR / "train",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode="int",
    shuffle=True,
    seed=42
)

class_names = train_ds_raw.class_names
print("Classes:", class_names)

# Load val and test datasets
val_ds_raw = image_dataset_from_directory(
    DATASET_DIR / "val",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode="int",
    shuffle=True,
    seed=42
)

test_ds_raw = image_dataset_from_directory(
    DATASET_DIR / "test",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode="int",
    shuffle=False
)


Found 43500 files belonging to 29 classes.
Classes: ['Apple___Apple_Scab', 'Apple___Black_Rot', 'Apple___Cedar_Apple_Rust', 'Apple___Healthy', 'Bell Pepper___Bacterial_Spot', 'Bell Pepper___Healthy', 'Cherry___Healthy', 'Cherry___Powdery_Mildew', 'Corn (Maize)___Cercospora_Leaf_Spot', 'Corn (Maize)___Common_Rust_', 'Corn (Maize)___Healthy', 'Corn (Maize)___Northern_Leaf_Blight', 'Grape___Black_Rot', 'Grape___Esca_(Black_Measles)', 'Grape___Healthy', 'Grape___Leaf_Blight', 'Peach___Bacterial_Spot', 'Peach___Healthy', 'Potato___Early_Blight', 'Potato___Healthy', 'Potato___Late_Blight', 'Strawberry___Healthy', 'Strawberry___Leaf_Scorch', 'Tomato___Bacterial_Spot', 'Tomato___Early_Blight', 'Tomato___Healthy', 'Tomato___Late_Blight', 'Tomato___Septoria_Leaf_Spot', 'Tomato___Yellow_Leaf_Curl_Virus']
Found 6525 files belonging to 29 classes.
Found 6525 files belonging to 29 classes.


In [64]:
# Preprocess images for MobileNetV3
train_ds = train_ds_raw.map(lambda x, y: (preprocess_input(x), y))
val_ds = val_ds_raw.map(lambda x, y: (preprocess_input(x), y))
test_ds = test_ds_raw.map(lambda x, y: (preprocess_input(x), y))

# Optimize performance
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.prefetch(buffer_size=AUTOTUNE)


In [65]:
# Base model with pretrained weights, without top classifier
base_model = MobileNetV3Large(
    input_shape=(*IMG_SIZE, 3),
    include_top=False,
    weights="imagenet"
)
base_model.trainable = False  # Freeze initially

# Custom classifier on top
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.3)(x)
output = layers.Dense(len(class_names), activation="softmax")(x)

model = models.Model(inputs=base_model.input, outputs=output)

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_3         │ (None, 224, 224,  │          0 │ input_layer_3[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv (Conv2D)       │ (None, 112, 112,  │        432 │ rescaling_3[0][0] │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv_bn             │ (None, 112, 112,  │         64 │ conv[0][0]        │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_60       │ (None, 112, 112,  │          0 │ conv_bn[0][0]     │
│ (Activation)        │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        144 │ activation_60[0]… │
│ (DepthwiseConv2D)   │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │         64 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_57 (ReLU)     │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        256 │ re_lu_57[0][0]    │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_add   │ (None, 112, 112,  │          0 │ activation_60[0]… │
│ (Add)               │ 16)               │            │ expanded_conv_pr… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_1_ex… │ (None, 112, 112,  │      1,024 │ expanded_conv_ad… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_1_ex… │ (None, 112, 112,  │        256 │ expanded_conv_1_… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_58 (ReLU)     │ (None, 112, 112,  │          0 │ expanded_conv_1_… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_1_de… │ (None, 113, 113,  │          0 │ re_lu_58[0][0]    │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_1_de… │ (None, 56, 56,    │        576 │ expanded_conv_1_… │
│ (DepthwiseConv2D)   │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_1_de… │ (None, 56, 56,    │        256 │ expanded_conv_1_

 Total params: 3,123,101 (11.91 MB)

 Trainable params: 126,749 (495.11 KB)

 Non-trainable params: 2,996,352 (11.43 MB)

In [66]:
early_stop = EarlyStopping(
    monitor="val_loss",
    patience=5,
    restore_best_weights=True
)

checkpoint = ModelCheckpoint(
    "best_model.h5",
    monitor="val_accuracy",
    save_best_only=True,
    verbose=1
)


In [67]:
EPOCHS = 20

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=[early_stop, checkpoint]
)


Epoch 1/20
680/680 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step - accuracy: 0.7428 - loss: 0.9505
Epoch 1: val_accuracy improved from -inf to 0.97195, saving model to best_model.h5


680/680 ━━━━━━━━━━━━━━━━━━━━ 104s 123ms/step - accuracy: 0.7430 - loss: 0.9498 - val_accuracy: 0.9720 - val_loss: 0.0932
Epoch 2/20
679/680 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - accuracy: 0.9597 - loss: 0.1314
Epoch 2: val_accuracy improved from 0.97195 to 0.98038, saving model to best_model.h5


680/680 ━━━━━━━━━━━━━━━━━━━━ 57s 84ms/step - accuracy: 0.9597 - loss: 0.1313 - val_accuracy: 0.9804 - val_loss: 0.0611
Epoch 3/20
679/680 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - accuracy: 0.9728 - loss: 0.0841
Epoch 3: val_accuracy improved from 0.98038 to 0.98253, saving model to best_model.h5


680/680 ━━━━━━━━━━━━━━━━━━━━ 77s 77ms/step - accuracy: 0.9728 - loss: 0.0841 - val_accuracy: 0.9825 - val_loss: 0.0473
Epoch 4/20
679/680 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - accuracy: 0.9806 - loss: 0.0624
Epoch 4: val_accuracy improved from 0.98253 to 0.98498, saving model to best_model.h5


680/680 ━━━━━━━━━━━━━━━━━━━━ 52s 77ms/step - accuracy: 0.9806 - loss: 0.0624 - val_accuracy: 0.9850 - val_loss: 0.0417
Epoch 5/20
680/680 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - accuracy: 0.9830 - loss: 0.0527
Epoch 5: val_accuracy improved from 0.98498 to 0.98774, saving model to best_model.h5


680/680 ━━━━━━━━━━━━━━━━━━━━ 92s 92ms/step - accuracy: 0.9830 - loss: 0.0527 - val_accuracy: 0.9877 - val_loss: 0.0326
Epoch 6/20
679/680 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - accuracy: 0.9862 - loss: 0.0425
Epoch 6: val_accuracy did not improve from 0.98774
680/680 ━━━━━━━━━━━━━━━━━━━━ 52s 76ms/step - accuracy: 0.9862 - loss: 0.0425 - val_accuracy: 0.9873 - val_loss: 0.0338
Epoch 7/20
679/680 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - accuracy: 0.9882 - loss: 0.0365
Epoch 7: val_accuracy improved from 0.98774 to 0.98820, saving model to best_model.h5


680/680 ━━━━━━━━━━━━━━━━━━━━ 52s 77ms/step - accuracy: 0.9882 - loss: 0.0365 - val_accuracy: 0.9882 - val_loss: 0.0336
Epoch 8/20
679/680 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - accuracy: 0.9884 - loss: 0.0354
Epoch 8: val_accuracy improved from 0.98820 to 0.99034, saving model to best_model.h5


680/680 ━━━━━━━━━━━━━━━━━━━━ 57s 83ms/step - accuracy: 0.9884 - loss: 0.0354 - val_accuracy: 0.9903 - val_loss: 0.0254
Epoch 9/20
679/680 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - accuracy: 0.9875 - loss: 0.0333
Epoch 9: val_accuracy did not improve from 0.99034
680/680 ━━━━━━━━━━━━━━━━━━━━ 80s 80ms/step - accuracy: 0.9875 - loss: 0.0333 - val_accuracy: 0.9884 - val_loss: 0.0297
Epoch 10/20
679/680 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - accuracy: 0.9891 - loss: 0.0325
Epoch 10: val_accuracy did not improve from 0.99034
680/680 ━━━━━━━━━━━━━━━━━━━━ 53s 78ms/step - accuracy: 0.9891 - loss: 0.0325 - val_accuracy: 0.9882 - val_loss: 0.0307
Epoch 11/20
679/680 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - accuracy: 0.9908 - loss: 0.0270
Epoch 11: val_accuracy did not improve from 0.99034
680/680 ━━━━━━━━━━━━━━━━━━━━ 81s 77ms/step - accuracy: 0.9908 - loss: 0.0270 - val_accuracy: 0.9864 - val_loss: 0.0357
Epoch 12/20
680/680 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - accuracy: 0.9907 - loss: 0.0260
Epoch 12: val_accu

In [69]:
# Evaluate model on the test dataset
loss, accuracy = model.evaluate(test_ds)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")


102/102 ━━━━━━━━━━━━━━━━━━━━ 9s 84ms/step - accuracy: 0.9928 - loss: 0.0204
Test Loss: 0.0312
Test Accuracy: 0.9887


In [70]:
# Unfreeze the base model
base_model.trainable = True

# Optional: Freeze the first few layers if you want to prevent overfitting
for layer in base_model.layers[:100]:
    layer.trainable = False

# Compile again with a lower learning rate
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

# Continue training for a few epochs
history = model.fit(train_ds,
                    validation_data=val_ds,
                    epochs=5)  # small number of fine-tuning epochs


Epoch 1/5
680/680 ━━━━━━━━━━━━━━━━━━━━ 121s 129ms/step - accuracy: 0.9361 - loss: 0.2428 - val_accuracy: 0.9640 - val_loss: 0.1188
Epoch 2/5
680/680 ━━━━━━━━━━━━━━━━━━━━ 63s 92ms/step - accuracy: 0.9935 - loss: 0.0207 - val_accuracy: 0.9920 - val_loss: 0.0225
Epoch 3/5
680/680 ━━━━━━━━━━━━━━━━━━━━ 61s 90ms/step - accuracy: 0.9974 - loss: 0.0080 - val_accuracy: 0.9957 - val_loss: 0.0122
Epoch 4/5
680/680 ━━━━━━━━━━━━━━━━━━━━ 82s 91ms/step - accuracy: 0.9981 - loss: 0.0063 - val_accuracy: 0.9969 - val_loss: 0.0122
Epoch 5/5
680/680 ━━━━━━━━━━━━━━━━━━━━ 81s 90ms/step - accuracy: 0.9978 - loss: 0.0070 - val_accuracy: 0.9966 - val_loss: 0.0119


In [71]:
test_loss, test_acc = model.evaluate(test_ds)
print(f"Test Accuracy: {test_acc:.4f}, Test Loss: {test_loss:.4f}")


102/102 ━━━━━━━━━━━━━━━━━━━━ 7s 70ms/step - accuracy: 0.9980 - loss: 0.0078
Test Accuracy: 0.9971, Test Loss: 0.0119


In [73]:
import json
import numpy as np
from sklearn.metrics import classification_report

# Load class names from JSON
with open("/content/plant_classes.json", "r") as f:
    class_names = json.load(f)

# Extract true labels from the test dataset
y_true = np.concatenate([y.numpy() for x, y in test_ds], axis=0)

# Predict labels
y_pred = np.argmax(model.predict(test_ds), axis=1)

# Generate classification report
print(classification_report(y_true, y_pred, target_names=class_names))


102/102 ━━━━━━━━━━━━━━━━━━━━ 7s 70ms/step
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       225
           1       1.00      1.00      1.00       225
           2       1.00      1.00      1.00       225
           3       1.00      1.00      1.00       225
           4       1.00      1.00      1.00       225
           5       1.00      1.00      1.00       225
           6       0.99      1.00      1.00       225
           7       1.00      1.00      1.00       225
           8       0.99      0.98      0.98       225
           9       1.00      1.00      1.00       225
          10       1.00      1.00      1.00       225
          11       0.98      0.99      0.98       225
          12       1.00      1.00      1.00       225
          13       1.00      1.00      1.00       225
          14       1.00      1.00      1.00       225
          15       1.00      1.00      1.00       225
          16       1.00      1.00      

In [74]:
# Native Keras format (recommended)
model.save("/content/plant_disease_model.keras")
print("Saved in native Keras format (.keras)")


Saved in native Keras format (.keras)


In [75]:
# HDF5 format (legacy)
model.save("/content/plant_disease_model.h5")
print("Saved in HDF5 format (.h5)")


Saved in HDF5 format (.h5)


In [79]:
model.export("/content/plant_disease_model_savedmodel")


Saved artifact at '/content/plant_disease_model_savedmodel'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='keras_tensor_620')
Output Type:
  TensorSpec(shape=(None, 29), dtype=tf.float32, name=None)
Captures:
  139395913365840: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139395913361232: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139395913366032: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139395913357968: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139395913362768: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139395913366800: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139395913359504: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139395913353360: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139395913357584: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139395913367376: TensorSpec(shape=(), dtype=tf.resource,

In [80]:
converter = tf.lite.TFLiteConverter.from_saved_model("/content/plant_disease_model_savedmodel")
tflite_model = converter.convert()

with open("/content/model_from_savedmodel.tflite", "wb") as f:
    f.write(tflite_model)


In [3]:
import tensorflow as tf

interpreter = tf.lite.Interpreter(model_path="model_from_savedmodel.tflite")
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print("Input:", input_details)
print("Output:", output_details)


ModuleNotFoundError: No module named 'tensorflow'